In [1]:
import sys
import os
from tempfile import mkdtemp
sys.path.insert(0, '..')

import numpy as np
from process_bigraph import Composite, pp
from process_bigraph.experiments.parameter_scan import RunProcess

from biosimulator_processes import CORE 


Smoldyn is not properly installed in this environment and thus its process implementation cannot be registered. Please consult smoldyn documentation.
Available processes:
[ 'console-emitter',
  'ram-emitter',
  'composite',
  'cobra-process',
  'copasi-process',
  'tellurium-process',
  'utc-amici',
  'utc-copasi',
  'utc-tellurium',
  'copasi-step',
  'tellurium-step',
  'amici-step',
  'plotter',
  'plotter2d',
  'utc-comparator',
  'simple-neuron',
  'simple-neuron-network']


1. Define inputs

In [2]:
omex_dir = '../test_suite/examples/sbml-core/Vilar-PNAS-2002-minimal-circardian-clock-discrete-SSA'  #  Elowitz-Nature-2000-Repressilator'
omex_fp = omex_dir + '.omex'
sbml_fp = os.path.join(omex_dir, 'BIOMD0000000012_url.xml')
sedml_fp = os.path.join(omex_dir, 'simulation.sedml')
reports_path = os.path.join(omex_dir, 'reports.h5')
simulators = ['amici', 'copasi', 'tellurium']

In [11]:
from biosimulator_processes.helpers import check_ode_kisao_term, calc_step_size
from biosimulator_processes.io import get_sedml_time_config
def _get_sedml_time_params(omex_path: str):
    sedml_fp = None
    for f in os.listdir(omex_path):
        if f.endswith('.sedml'):
            sedml_fp = os.path.join(omex_path, f)
    assert sedml_fp is not None, 'Your OMEX archive must contain a valid SEDML file.'
    # sedml_fp = os.path.join(omex_path, 'simulation.sedml')
    sedml_utc_config = get_sedml_time_config(sedml_fp)
    output_end = int(sedml_utc_config['outputEndTime'])
    output_start = int(sedml_utc_config['outputStartTime'])
    duration = output_end - output_start
    n_steps = int(sedml_utc_config['numberOfPoints'])
    # check kisao id for supported algorithm/kisao ID
    specified_alg = sedml_utc_config['algorithm'].split(':')[1]
    supported_alg = check_ode_kisao_term(specified_alg)
    if not supported_alg:
        raise ValueError('Algorithm specified in OMEX archive is non-deterministic and thus not supported by a Uniform Time Course implementation.')
    return {
        'duration': output_end,  # duration,
        'num_steps': n_steps + 1,  # to account for self comparison
        'step_size': calc_step_size(duration, n_steps),
        'output_start_time': output_start,
        'initial_time': int(sedml_utc_config['initialTime']),
    }


time = _get_sedml_time_params(omex_dir)

time

{'duration': 1000,
 'num_steps': 601,
 'step_size': 1.0,
 'output_start_time': 400,
 'initial_time': 0,
 'alg': '0000019',
 'supported': True}

In [8]:
time['alg'].split(':')[1]

'0000019'

2. Define composition document (manuscript?)

In [3]:
manuscript = {
    'amici': {
        '_type': 'step',
        'address': 'local:utc-amici',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['amici_floating_species_store'],
            'model_parameters': ['amici_model_parameters_store'],
            'reactions': ['amici_reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['amici_floating_species_store']
        }
    },
    'copasi': {
        '_type': 'step',
        'address': 'local:utc-copasi',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['copasi_floating_species_store'],
            'model_parameters': ['copasi_model_parameters_store'],
            'reactions': ['copasi_reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['copasi_floating_species_store']
        }
    },
    'tellurium': {
        '_type': 'step',
        'address': 'local:utc-tellurium',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['tellurium_floating_species_store'],
            'model_parameters': ['tellurium_model_parameters_store'],
            'reactions': ['tellurium_reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['tellurium_floating_species_store']
        }
    },
    'comparison': {
        '_type': 'step',
        'address': 'local:utc-comparator',
        'config': {
            'simulators': ['amici', 'copasi', 'tellurium'],
        },
        'inputs': {
            'time': ['time_store'],
            'amici_floating_species': ['amici_floating_species_store'],
            'copasi_floating_species': ['copasi_floating_species_store'],
            'tellurium_floating_species': ['tellurium_floating_species_store'],
        },
        'outputs': {
            'results': ['results_store'],
            'id': ['id_store']
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'results': 'tree[float]',
                'id': 'string'
            }
        },
        'inputs': {
            'results': ['results_store'],
            'id': ['id_store']
        },
        # 'outputs': {
        #     'results': ['results_store'],
        #     'id': ['id_store']
        # }
    }
}


output_bridge = {
    'inputs': {
        'results': ['results_store'],
        'id': ['id_store']
    },
    'outputs': {
        'results': ['results_store'],
        'id': ['id_store']
    }
}

3. Create composition with doc specifications

In [4]:
composition_config = {'state': manuscript}  #, 'bridge': output_bridge}
composition = Composite(
    config=composition_config,
    core=CORE
)

2024-06-20 07:12:56.607 - amici.sbml_import - INFO - Finished importing SBML                         (4.21E-02s)
pyNeuroML >>> INFO - Finished importing SBML                         (4.21E-02s)
2024-06-20 07:12:56.655 - amici.sbml_import - INFO - Finished processing SBML observables            (4.35E-02s)
pyNeuroML >>> INFO - Finished processing SBML observables            (4.35E-02s)
2024-06-20 07:12:56.660 - amici.sbml_import - INFO - Finished processing SBML event observables      (8.33E-07s)
pyNeuroML >>> INFO - Finished processing SBML event observables      (8.33E-07s)
2024-06-20 07:12:56.687 - amici.de_model - INFO - Finished computing xdot                            (3.82E-03s)
pyNeuroML >>> INFO - Finished computing xdot                            (3.82E-03s)
2024-06-20 07:12:56.694 - amici.de_model - INFO - Finished computing x0                              (3.42E-03s)
pyNeuroML >>> INFO - Finished computing x0                              (3.42E-03s)
2024-06-20 07:12:56.713 

In [5]:
composition.run(1)

In [6]:
composition_results = composition.gather_results()

In [7]:
composition_results

{('emitter',): []}

In [8]:
composition.state

{'time_store': [],
 'amici_floating_species_store': {'PX': 0.0,
  'PY': 0.0,
  'PZ': 0.0,
  'X': 0.0,
  'Y': 20.0,
  'Z': 0.0},
 'amici_model_parameters_store': {'beta': 0.2,
  'alpha0': 0.2164,
  'alpha': 216.404,
  'translation efficiency': 20.0,
  'n': 2.0,
  'KM': 40.0,
  'mRNA half life': 2.0,
  'protein half life': 10.0,
  'average mRNA life time': 0.0,
  'kd_mRNA': 0.0,
  'kd_prot': 0.0,
  'k_tl': 0.0,
  'a_tr': 0.0,
  'tps_active': 0.5,
  'tps_repr': 0.0005,
  'a0_tr': 0.0},
 'amici_reactions_store': [],
 'copasi_floating_species_store': {'PX': 0.0,
  'PY': 0.0,
  'PZ': 0.0,
  'X': 0.0,
  'Y': 20.0,
  'Z': 0.0},
 'copasi_model_parameters_store': {'beta': 0.0,
  'alpha0': 0.0,
  'alpha': 0.0,
  'translation efficiency': 0.0,
  'n': 0.0,
  'KM': 0.0,
  'mRNA half life': 0.0,
  'protein half life': 0.0,
  'average mRNA life time': 0.0,
  'kd_mRNA': 0.0,
  'kd_prot': 0.0,
  'k_tl': 0.0,
  'a_tr': 0.0,
  'tps_active': 0.0,
  'tps_repr': 0.0,
  'a0_tr': 0.0},
 'copasi_reactions_store

In [3]:
from biosimulator_processes.execute import exec_utc_comparison
utc_comparison_result = exec_utc_comparison(omex_fp, simulators)


utc_comparison_result

ValueError: Algorithm specified in OMEX archive is non-deterministic and thus not supported by a Uniform Time Course implementation.

In [14]:

_c = utc_comparison_result.copy()
for spec_name, result in utc_comparison_result['results'].items():
    _c['results'][spec_name].pop('output_data')
pp(_c)

2024-06-11 12:32:42.854 - amici.sbml_import - INFO - Finished loading SBML                           (6.62E-03s)
2024-06-11 12:32:42.897 - amici.sbml_import - INFO - Finished importing SBML                         (3.85E-02s)
2024-06-11 12:32:42.913 - amici.sbml_import - INFO - Finished processing SBML observables            (1.28E-02s)
2024-06-11 12:32:42.917 - amici.sbml_import - INFO - Finished processing SBML event observables      (7.08E-07s)
2024-06-11 12:32:42.938 - amici.de_model - INFO - Finished computing xdot                            (3.34E-03s)
2024-06-11 12:32:42.944 - amici.de_model - INFO - Finished computing x0                              (2.71E-03s)
2024-06-11 12:32:42.954 - amici.de_model - INFO - Finished computing w                               (7.23E-03s)
2024-06-11 12:32:43.529 - amici.de_export - INFO - Finished generating cpp code                      (5.71E-01s)
2024-06-11 12:32:51.087 - amici.de_export - INFO - Finished compiling cpp code                  

{ 'id': 'utc-comparison',
  'results': { 'LacI mRNA': { 'mse': { 'amici': { 'amici': 0.0,
                                                  'copasi': 1.0884950422811668,
                                                  'tellurium': 305.76232201739214},
                                       'copasi': { 'amici': 1.0884950422811668,
                                                   'copasi': 0.0,
                                                   'tellurium': 307.2027588807725},
                                       'tellurium': { 'amici': 305.76232201739214,
                                                      'copasi': 307.2027588807725,
                                                      'tellurium': 0.0}},
                              'prox': { 'amici': { 'amici': True,
                                                   'copasi': False,
                                                   'tellurium': False},
                                        'copasi': { 'amici': False,
  

In [15]:
utc_comparison_result['id']

'utc-comparison'

In [16]:
from biosimulator_processes.api.instance import UtcComparisonDocument
doc = UtcComparisonDocument(model_source=omex_fp)

In [17]:
doc

{'state': {'amici': {'_type': 'step',
   'address': 'local:utc-amici',
   'config': {'model': {'model_source': '../test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator.omex'}},
   'inputs': {'time': ['time_store'],
    'floating_species': ['amici_floating_species_store'],
    'model_parameters': ['amici_model_parameters_store'],
    'reactions': ['amici_reactions_store']},
   'outputs': {'time': ['time_store'],
    'floating_species': ['amici_floating_species_store']}},
  'copasi': {'_type': 'step',
   'address': 'local:utc-copasi',
   'config': {'model': {'model_source': '../test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator.omex'}},
   'inputs': {'time': ['time_store'],
    'floating_species': ['copasi_floating_species_store'],
    'model_parameters': ['copasi_model_parameters_store'],
    'reactions': ['copasi_reactions_store']},
   'outputs': {'time': ['time_store'],
    'floating_species': ['copasi_floating_species_store']}},
  'tellurium': {'_type': 'step'

Perform an entire sbml source directory scan with the comparison:

In [ ]:
from biosimulator_processes.execute import sbml_utc_comparison_scan

source_dir = '../test_suite/examples/sbml-core'
scan_results = sbml_utc_comparison_scan(source_dir, simulators)

# successfully run
scan_results.keys()

2024-06-11 12:32:51.319 - amici.sbml_import - INFO - Finished loading SBML                           (1.19E-02s)
2024-06-11 12:32:51.607 - amici.sbml_import - INFO - Finished loading SBML                           (9.30E-03s)
2024-06-11 12:32:51.666 - amici.sbml_import - INFO - Finished importing SBML                         (5.63E-02s)
2024-06-11 12:32:51.667 - amici.sbml_import - WARNING - Conservation laws are currently not supported for models with events, piecewise or Heaviside functions, and will be turned off.
2024-06-11 12:32:51.706 - amici.sbml_import - INFO - Finished processing SBML observables            (3.63E-02s)
2024-06-11 12:32:51.710 - amici.sbml_import - INFO - Finished processing SBML event observables      (2.92E-07s)
2024-06-11 12:32:51.739 - amici.de_model - INFO - Finished computing xdot                            (4.41E-03s)
2024-06-11 12:32:51.745 - amici.de_model - INFO - Finished computing x0                              (2.69E-03s)
2024-06-11 12:32:51.760 -

In [ ]:
scan_results['Parmar-BMC-Syst-Biol-2017-iron-distribution.omex']

In [ ]:
scan_results['Szymanska-J-Theor-Biol-2009-HSP-synthesis.omex']

In [ ]:
for f in scan_results.keys():
    if not 'error' in scan_results[f].keys():
        print(f)

In [ ]:
from biosimulator_processes.execute import exec_utc_comparison

varusai_archive = os.path.join(source_dir, 'Varusai-Sci-Rep-2018-mTOR-signaling-LSODA-LSODAR-SBML.omex')
caravagna_utc_comparison = exec_utc_comparison(varusai_archive, simulators)

In [ ]:
scan_results['Varusai-Sci-Rep-2018-mTOR-signaling-LSODA-LSODAR-SBML.omex']

In [ ]:
amici_instance = composition.state['amici']['instance']

amici_result = amici_instance.update(amici_instance.initial_state())

In [ ]:
amici_instance.plot_results()

In [ ]:
emitter = composition.state['emitter']['instance']

In [ ]:
dir(emitter)

In [ ]:
emitter.inputs()

In [ ]:
update = emitter.invoke(composition.state['amici']['instance'].initial_state())

In [ ]:
update.__dict__

In [ ]:
shared_store_composition_doc = {
    'amici': {
        '_type': 'step',
        'address': 'local:utc-amici',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['floating_species_store'],
            'model_parameters': ['model_parameters_store'],
            'reactions': ['reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['floating_species_store']
        }
    },
    'copasi': {
        '_type': 'step',
        'address': 'local:utc-copasi',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['floating_species_store'],
            'model_parameters': ['model_parameters_store'],
            'reactions': ['reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['floating_species_store']
        }
    },
    'tellurium': {
        '_type': 'step',
        'address': 'local:utc-tellurium',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['floating_species_store'],
            'model_parameters': ['model_parameters_store'],
            'reactions': ['reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['floating_species_store']
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'time': 'list[float]',
                'floating_species': 'tree[float]',
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['floating_species_store']
        }
    }
}


shared_store_composition = Composite(config={'state': shared_store_composition_doc}, core=CORE)

In [ ]:
shared_store_composition.run(10)

In [ ]:
shared_res = shared_store_composition.gather_results()

In [ ]:
shared_res

In [ ]:
from process_bigraph.experiments.growth_division import grow_divide_agent, Grow, Divide


CORE.process_registry.register('grow', Grow)
CORE.process_registry.register('divide', Divide)

def test_grow_divide(core):
    initial_mass = 1.0

    grow_divide = grow_divide_agent(
        {'grow': {'rate': 0.03}},
        {'mass': initial_mass},
        ['environment', '0'])

    environment = {
        'environment': {
            '0': {
                'mass': initial_mass,
                'grow_divide': grow_divide}}}

    composite = Composite({
        'state': environment},
        core=core)

    updates = composite.update({}, 100.0)
    assert '0_0_0_0_0' in composite.state['environment']
    return updates 

In [ ]:
grow_results = test_grow_divide(CORE)

In [ ]:
grow_results

In [ ]:
from process_bigraph.experiments.parameter_scan import test_parameter_scan


def test_parameter_scan():
    # TODO: make a parameter scan with a biosimulator process,
    #   ie - Copasi

    state = {
        'scan': {
            '_type': 'step',
            'address': 'local:!process_bigraph.experiments.parameter_scan.ParameterScan',
            'config': {
                'parameter_ranges': [(
                    ['rates', 'A', 'kdeg'], [0.0, 0.1, 1.0, 10.0])],
                'process_address': 'local:!process_bigraph.experiments.parameter_scan.ToySystem',
                'process_config': {
                    'rates': {
                        'A': {
                            'ksynth': 1.0}}},
                'observables': [
                    ['species', 'A']],
                'initial_state': {
                    'species': {
                        'A': 13.3333}},
                'timestep': 1.0,
                'runtime': 10},
            'outputs': {
                'results': ['results']}}}

    # TODO: make a Workflow class that is a Step-composite
    # scan = Workflow({
    scan = Composite({
        'bridge': {
            'outputs': {
                'results': ['results']}},
        'state': state})
            
    # TODO: make a method so we can run it directly, provide some way to get the result out
    # result = scan.update({})
    result = scan.update({}, 0.0)
    return result


In [ ]:
shared_store_composition = Composite({
    'state': shared_store_composition_doc,
    'bridge': {
        'outputs': {
            'floating_species': ['floating_species_store']
        }
    }
}, CORE)

In [ ]:
result = shared_store_composition.update(shared_store_composition.state, 1.0)

result

In [ ]:
test_parameter_scan()

In [ ]:
composition.update({}, 0.0)

In [ ]:
composition.config

In [ ]:
composition.run(10)

In [ ]:
composition.steps_run

In [ ]:
c = Composite(config={'state': manuscript, 'bridge': output_bridge}, core=CORE)

In [ ]:
c.bridge

In [ ]:
c.run(1)

In [ ]:
c.gather_results()

In [ ]:
x = {'outputEndTime': '100.0'}

end = int(float(x['outputEndTime']))

In [4]:
from biosimulator_processes.execute import exec_utc_comparison

In [5]:
c = exec_utc_comparison(omex_fp, simulators, 'comparison', False)

2024-06-11 12:33:47.337 - amici.sbml_import - INFO - Finished importing SBML                         (3.70E-02s)
2024-06-11 12:33:47.385 - amici.sbml_import - INFO - Finished processing SBML observables            (4.46E-02s)
2024-06-11 12:33:47.389 - amici.sbml_import - INFO - Finished processing SBML event observables      (5.00E-07s)
2024-06-11 12:33:47.412 - amici.de_model - INFO - Finished computing xdot                            (3.10E-03s)
2024-06-11 12:33:47.418 - amici.de_model - INFO - Finished computing x0                              (2.91E-03s)
2024-06-11 12:33:47.438 - amici.de_model - INFO - Finished computing w                               (1.61E-02s)
2024-06-11 12:33:48.108 - amici.de_export - INFO - Finished generating cpp code                      (6.65E-01s)
2024-06-11 12:33:56.815 - amici.de_export - INFO - Finished compiling cpp code                       (8.70E+00s)


In [6]:
c['results']

{'LacI protein': {'mse': {'amici': {'amici': 0.0,
    'copasi': 3421.101751414077,
    'tellurium': 1112642.3515076651},
   'copasi': {'amici': 3421.101751414077,
    'copasi': 0.0,
    'tellurium': 1122517.493697266},
   'tellurium': {'amici': 1112642.3515076651,
    'copasi': 1122517.493697266,
    'tellurium': 0.0}},
  'prox': {'amici': {'amici': True, 'copasi': False, 'tellurium': False},
   'copasi': {'amici': False, 'copasi': True, 'tellurium': False},
   'tellurium': {'amici': False, 'copasi': False, 'tellurium': True}}},
 'TetR protein': {'mse': {'amici': {'amici': 0.0,
    'copasi': 3432.245484968864,
    'tellurium': 1182387.32612705},
   'copasi': {'amici': 3432.245484968864,
    'copasi': 0.0,
    'tellurium': 1178485.6823350259},
   'tellurium': {'amici': 1182387.32612705,
    'copasi': 1178485.6823350259,
    'tellurium': 0.0}},
  'prox': {'amici': {'amici': True, 'copasi': False, 'tellurium': False},
   'copasi': {'amici': False, 'copasi': True, 'tellurium': False},
   '